<img src="https://images.pexels.com/photos/248152/pexels-photo-248152.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260" style='width:300px'>

## Classification Case Study

The researchers at SABIC Innovative Plastics US are working on refining the manufacturing process for new polymers. Creating this polymer involves combining 12 materials and 45 mechanical processes (order of processes is unimportant). The prices of ingredients vary as do the amount of time needed for each process done by individual machines that would otherwise be used to make other products. The CSuite requires a pared down list of materials and processes before polymers can go to market. 

A polymer yield greater than or equal to 41 grams is considered "high".

You will use logistic regression to create a classification model find the material and process combination that produces high yield.

### Business Problem

#### End Users:

- Scientists, budgeting department, CSuite

#### True business problem:

- Create a model that can, if given data from from repeated chemical manufacturing trials, predict if the polymer yield will be high (worth the money). 

#### Context:

- **False negative** in this context: Predicts that an actually high-yield combination will be low yield.
    - **Outcome**: Missed polymer opportunity
    
- **False positive** in this context: Predicts that an actually low-yield combination will be high yield.

    - **Outcome**: Unreliable formula. Takes longer to go to market. Potential company waste.

#### Evaluation 

Which metric (of the ones we've explore so far) would make sense to primarily use as we evaluate our models?

- **Accuracy**
- Precision
- Recall
- F1-Score

## Import necessary packages

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

from IPython.display import display, HTML

## Read in data & take a look

In [ ]:
df = pd.read_csv('ChemicalManufacturingProcess.csv')

In [ ]:
df.info()

In [ ]:
df.head()

## Train test split

Encode `Yield` into `1/0` depending on if it's equal or greater than 41.

In [ ]:
# train test split

X = df.drop(columns = ['Yield'])
y = (df['Yield'] >= 41).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Preprocessing pipeling
- impute missing values in columns
- scale using max absolute value (this was chosen after trial and error with other scalers)

In [ ]:
# Pipeline preprocessing
imp = SimpleImputer(missing_values=np.nan, strategy='median')
mscale = MaxAbsScaler()

prep = [('imputer', imp), ('scaler', mscale)]

pipe = Pipeline(prep)

In [ ]:
pipe.fit(X_train)

In [ ]:
X_train_prepped = pipe.transform(X_train)

## Create logistic regression model
Using *all* variables. 

In [ ]:
lgrg1 = LogisticRegression(random_state = 100)

In [ ]:
lgrg1.fit(X_train_prepped, y_train)

In [ ]:
y_pred = lgrg1.predict(X_train_prepped)

## Evaluate
Check the accuracy of the model. <br>
Remember, `sklearn` uses a threshold cutoff of `0.5`

In [ ]:
accuracy_score(y_train, y_pred)

## But is that the **BEST** cutoff?

- get predicted probabilities using `X_train_prepped`
- use `pipe` to transform `X_test`
- predict `y_test_pred_probs` using `lgrg1`

In [ ]:
y_train_pred_probs = lgrg1.predict_proba(X_train_prepped)[:, 1]

In [ ]:
# Transform X_test
X_test_transformed = pipe.transform(X_test)

# Get probabilites
y_test_pred_probs = lgrg1.predict_proba(X_test_transformed)[:, 1]

Use `for` loop to iterate over various thresholds and calculate the accuracy for both train and test.

In [ ]:
metrics_df = pd.DataFrame(columns=['threshold', 'train_acc', 'test_acc'])

for x in np.arange(y_train_pred_probs.min(), y_train_pred_probs.max(), 0.01):
    placeholder = []
    threshold = x
    y_pred_train = np.where(y_train_pred_probs > x, 1, 0)
    r_pred_test = np.where(y_test_pred_probs > x, 1, 0)
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, r_pred_test)
    placeholder = [threshold, train_acc, test_acc]
    sample_S = pd.Series(placeholder, index=[
                         'threshold', 'train_acc', 'test_acc'])
    metrics_df = metrics_df.append(sample_S, ignore_index=True)

## **Plot** the results

In [ ]:
plt.figure(figsize=(13, 13))
plt.title("Search for Best Threshold using Accuracy",
          fontsize=16)

plt.xlabel("Threshold")
plt.ylabel("Score")
plt.grid(True)

ax = plt.gca()
ax.set_xlim(y_train_pred_probs.min(), y_train_pred_probs.max())
ax.set_ylim(0, 1)
ax.xaxis.set_major_locator(MaxNLocator(20))
ax.yaxis.set_major_locator(MaxNLocator(20))

# Get the regular numpy array from the MaskedArray
X_axis = np.array(metrics_df['threshold'].data, dtype=float)

ax.plot(X_axis, metrics_df['train_acc'], ls='--',
        color='g', label='Train Accuracy')
ax.plot(X_axis, metrics_df['test_acc'], ls='-',
        color='g', label='Test Accuracy')

plt.legend()

## Question:

- Is `.5` the best cutoff?
- What is the best choice?
- Why?

-----

## **Advanced** Questions (Optional):

- While you're able to predict high yield, are you able to produce a reduced process/materials list?
- How would you adjust the `LogisticRegression` arguments to produce a reduced list?
- Is the model "done"? How would you check to see if multi-collinearity is an issue?

In [ ]:
# Check Coefficients
coef_dict = {}
for coef, feat in zip(list(lgrg1.coef_[0]),X_train.columns.tolist()):
    coef_dict[feat] = coef
    
# Convert to dataframe so you can sort it
coef_df = pd.DataFrame.from_dict(coef_dict, columns =['Coef'], orient='index')
coef_df.sort_values(by=['Coef'], ascending = False)

#### Set `C` to 0.02 to increase the penalty and perhaps decrease the variable set

In [ ]:
lgrg2 = LogisticRegression(C=.02, random_state = 100)
lgrg2.fit(X_train_prepped, y_train)
y_train_pred_probs = lgrg2.predict_proba(X_train_prepped)[:, 1]
y_test_pred_probs = lgrg2.predict_proba(X_test_transformed)[:, 1]

#### Check - Did this change the thresholds?

In [ ]:
metrics_df = pd.DataFrame(columns = ['threshold','train_acc', 'test_acc'])

for x in np.arange(y_train_pred_probs.min(),y_train_pred_probs.max(), 0.01):
    placeholder =[]
    threshold = x
    y_pred_train = np.where(y_train_pred_probs>x,1,0)
    r_pred_test = np.where(y_test_pred_probs>x, 1, 0)
    train_acc = accuracy_score(y_train,y_pred_train)
    test_acc = accuracy_score(y_test,r_pred_test)
    placeholder = [threshold,train_acc, test_acc]
    sample_S = pd.Series(placeholder,index = ['threshold','train_acc', 'test_acc'] )
    metrics_df = metrics_df.append(sample_S, ignore_index = True)

In [ ]:
plt.figure(figsize=(13, 13))
plt.title("Search for Best Threshold When LASSO Penalty = 0.02",
          fontsize=16)

plt.xlabel("Threshold")
plt.ylabel("Score")
plt.grid(True)

ax = plt.gca()
ax.set_xlim(y_train_pred_probs.min(), y_train_pred_probs.max())
ax.set_ylim(0, 1)
ax.xaxis.set_major_locator(MaxNLocator(20))
ax.yaxis.set_major_locator(MaxNLocator(20))

# Get the regular numpy array from the MaskedArray
X_axis = np.array(metrics_df['threshold'].data, dtype=float)

ax.plot(X_axis, metrics_df['train_acc'], ls='--', color='g', label='Train Accuracy')
ax.plot(X_axis, metrics_df['test_acc'], ls='-', color='g', label='Test Accuracy')

plt.legend()

### How did it change the coefficients?

Did any of them move closer to zero so you can give a reduced list of variables to managment?

In [ ]:
# Check Coefficients
coef_dict = {}
for coef, feat in zip(list(lgrg2.coef_[0]),X_train.columns.tolist()):
    coef_dict[feat] = coef
    
# Convert to dataframe so you can sort it
coef_df2 = pd.DataFrame.from_dict(coef_dict, columns =['Coef2'], orient='index')
coef_df2.sort_values(by=['Coef2'], ascending = False)

coef_df.merge(coef_df2,left_index=True, right_index = True)

In [ ]:
coef_df.merge(coef_df2,left_index=True, right_index = True)

#### Check for multi-collinearity
(do not need to solve it now, only checking)

In [ ]:
df_corr = X_train.corr().stack().reset_index()

# rename the columns
df_corr.columns = ['FEATURE_1', 'FEATURE_2', 'CORRELATION']

# create a mask to identify rows with duplicate features as mentioned above
mask_dups = (df_corr[['FEATURE_1', 'FEATURE_2']].apply(frozenset, axis=1).duplicated()) | (df_corr['FEATURE_1']==df_corr['FEATURE_2']) 

# apply the mask to clean the correlation dataframe
df_corr = df_corr[~mask_dups]
df_corr[abs(df_corr['CORRELATION'])>.8]